# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [29]:
from keras.src.saving.saving_api import load_model
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
import sys
from collections import deque  

#graficos
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import io
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight

#voz
import pyttsx3
import threading

## MOTOR TEXTO A VOZ

In [30]:
# Inicializa el motor de texto-a-voz de pyttsx3
tts_engine = pyttsx3.init()  # Crea una instancia del motor TTS

# Crea un objeto de bloqueo para sincronización de hilos
tts_lock = threading.Lock()  # Previene acceso concurrente al motor TTS

# Variable global para almacenar la última seña vocalizada
last_spoken_gesture = None  # Guarda el texto del último gesto reproducido

def speak_text(text):
    global last_spoken_gesture  # Accede a la variable global
    
    # Bloquea el acceso concurrente usando with para manejo seguro del recurso
    with tts_lock:  # Asegura que solo un hilo use el motor TTS a la vez
        
        # Verifica si el texto es diferente al último reproducido
        if text != last_spoken_gesture:  # Evita repeticiones consecutivas
            
            # Actualiza el registro del último gesto vocalizado
            last_spoken_gesture = text  # Almacena el nuevo texto
            
            # Añade el texto a la cola de reproducción
            tts_engine.say(text)  # Programa la reproducción del texto
            
            # Ejecuta la reproducción y espera a que termine
            tts_engine.runAndWait()  # Bloquea hasta terminar la reproducción

## 2. INICIALIZAR MEDIAPIPE

In [31]:
# Configuración inicial global
mp_hands = mp.solutions.hands

# Optimizar MediaPipe
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.45,  # Reducir confianza
    min_tracking_confidence=0.45,
    model_complexity=0  # Menor complejidad
)

mp_draw = mp.solutions.drawing_utils
dataset_dir = "dataset_11_90"
dataset_dir_video = "dataset_11_90_videos"
model_path = "gesture_model_me_10_90.h5"
sequence_length = 90
total_landmarks = 126
gestures = []
X_mean = None
X_std = None

num_camara = 0

## 3. FUNCIONES PRINCIPALES

In [32]:
# Funciones principales
def init_system():
    global gestures
    os.makedirs(dataset_dir, exist_ok=True)
    gestures = get_existing_gestures()
    
def get_existing_gestures():
    return [d for d in os.listdir(dataset_dir) 
           if os.path.isdir(os.path.join(dataset_dir, d))]

## 4. DETECCION DE MANO

In [33]:
def detect_hands():
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Detección de Manos", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

## 5. RECOLLECION DE DATOS 

In [34]:
def collect_data():
    global gestures
    gesture = input("\nIngrese la palabra o letra para la cual desea recolectar datos: ").upper()
    num_sequences = int(input("Ingrese el número de secuencias a capturar (recomendado: 50): "))
    
    save_dir = os.path.join(dataset_dir, gesture)
    os.makedirs(save_dir, exist_ok=True)

    save_dir_video = os.path.join(dataset_dir_video, gesture)
    os.makedirs(save_dir_video, exist_ok=True)

    # Parámetros para la creación del video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Códec para el video
    fps = 20.0                               # Velocidad de fotogramas
    frame_size = (640, 480)                  # Tamaño de cada fotograma (ancho, alto)

    print(f"\nRecolectando datos para el gesto '{gesture}'. Presiona 'ESC' para cancelar.")
    print("Mantenga la seña frente a la cámara...")
    
    cap = cv2.VideoCapture(0)
    sequence = []    # Lista para almacenar los landmarks de la secuencia actual
    counter = 0      # Contador de secuencias capturadas

    # Inicializamos la variable para el VideoWriter (para el video de la secuencia actual)
    video_writer = None

    # Configurar ventana para visualizar los landmarks en tiempo real
    landmark_window_name = "Landmarks en Tiempo Real"
    cv2.namedWindow(landmark_window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(landmark_window_name, 640, 480)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Crear un canvas negro para dibujar los landmarks
        landmark_canvas = np.zeros((480, 640, 3), dtype=np.uint8)
        
        # Procesar la imagen para detectar manos (se asume que 'hands' está previamente configurado)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            all_landmarks = []
            
            # Dibujar landmarks en el canvas negro y extraer las coordenadas para el dataset
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(
                    landmark_canvas,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_draw.DrawingSpec(color=(0, 0, 255), thickness=2)
                )
                
                # Extraer las coordenadas (x, y, z) de cada punto
                for lm in hand_landmarks.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])
            
            # Si se detecta una sola mano, se rellena con ceros para que cada secuencia tenga la misma cantidad de datos
            if len(results.multi_hand_landmarks) < 2:
                all_landmarks += [0.0] * 63  # 21 puntos x 3 coordenadas = 63 valores
            
            sequence.append(all_landmarks)
            
            # También se dibujan los landmarks sobre el frame original para la visualización
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Si se inicia una nueva secuencia, se crea un VideoWriter para grabar este bloque de video
        if video_writer is None: #if len(sequence) == 0:
            video_filename_seq = os.path.join(save_dir_video, f"{gesture}_landmarks_seq_{counter}.avi")
            video_writer = cv2.VideoWriter(video_filename_seq, fourcc, fps, frame_size)
        
        # Escribir el canvas actual en el video de la secuencia
        video_writer.write(landmark_canvas)
        
        # Mostrar información de cuántas secuencias se han capturado
        info_text = f"Secuencias: {counter}/{num_sequences}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(landmark_canvas, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Mostrar las ventanas
        cv2.imshow("Recolección de Datos", frame)
        cv2.imshow(landmark_window_name, landmark_canvas)
        
        # Cuando se alcanza la longitud deseada de la secuencia, se guarda el array y se cierra el video actual
        if len(sequence) == sequence_length:
            np.save(os.path.join(save_dir, f"secuencia_{counter}.npy"), sequence)
            counter += 1
            sequence = []
            if video_writer is not None:
                video_writer.release()
                video_writer = None
            print(f"Secuencias capturadas: {counter}/{num_sequences}")

        # Salir si se presiona 'ESC' o se han capturado el número deseado de secuencias
        if cv2.waitKey(1) & 0xFF == 27 or counter >= num_sequences:
            break

    # Liberar recursos
    cap.release()
    if video_writer is not None:
        video_writer.release()
    cv2.destroyAllWindows()
    gestures = get_existing_gestures()
    print(f"\nSe recolectaron {counter} secuencias para el gesto '{gesture}'")


## 6. CARGA DE DATOS

In [35]:
def load_data():
    X = []
    y = []
    
    for label_idx, gesture in enumerate(gestures):
        gesture_dir = os.path.join(dataset_dir, gesture)
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]
        
        for seq_file in sequences:
            seq_path = os.path.join(gesture_dir, seq_file)
            sequence = np.load(seq_path)
            
            if sequence.shape == (sequence_length, total_landmarks):
                X.append(sequence)
                y.append(label_idx)
    
    return np.array(X), np.array(y), gestures


## 7. ENTRENAMIENTO DEL MODELO

In [36]:
def train_model():
    global X_mean, X_std, gestures
    
    # 1. Verificar datos de entrenamiento
    gestures = get_existing_gestures()
    if not gestures:
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return

    # 2. Cargar y preparar datos
    print("\nCargando datos y preparando el entrenamiento...")
    X, y, gestures = load_data()
    y = tf.keras.utils.to_categorical(y)

    # 3. Calcular parámetros de normalización
    X_mean = np.mean(X, axis=(0, 1))
    X_std = np.std(X, axis=(0, 1))
    X = (X - X_mean) / X_std  # Normalización

    # 4. Guardar parámetros de normalización
    np.savez('normalization_params_90.npz', mean=X_mean, std=X_std)
    
    # 5. Arquitectura optimizada del modelo
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))
    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)
    outputs = tf.keras.layers.Dense(len(gestures), activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # 6. Compilación y entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nIniciando entrenamiento...")
    history = model.fit(
        X, y,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )

    # Calcular pesos para clases desbalanceadas
    y_true = np.argmax(y, axis=1)
    class_weights = compute_class_weight("balanced", classes=np.unique(y_true), y=y_true)
    class_weights = dict(enumerate(class_weights))

     # Función para graficar métricas de entrenamiento
    def generate_training_plots(history):
        # Crear figura de 12x5 pulgadas (ancho x alto)
        plt.figure(figsize=(12, 5))
        
        # Primer subgráfico (1 fila, 2 columnas, posición 1)
        plt.subplot(1, 2, 1)
        # Graficar precisión de entrenamiento (datos del historial)
        plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
        # Graficar precisión de validación (línea punteada)
        plt.plot(history.history['val_accuracy'], '--', label='Precisión Validación')
        plt.title('Evolución de la Precisión')
        plt.ylabel('Precisión')  # Rango 0-1 (0% a 100%)
        plt.xlabel('Época')  # Iteración de entrenamiento
        plt.legend()  # Mostrar etiquetas
        plt.grid(True, linestyle='--', alpha=0.5)  # Cuadrícula suave

        # Segundo subgráfico (posición 2)
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
        plt.plot(history.history['val_loss'], '--', label='Pérdida Validación')
        plt.title('Evolución de la Pérdida')
        plt.ylabel('Pérdida')  # Valor de la función de costo
        plt.xlabel('Época')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)

        # Ajustar espacios entre subgráficos
        plt.tight_layout()
        # Guardar como imagen PNG (calidad vectorial)
        plt.savefig('training_metrics.png', dpi=300, bbox_inches='tight')
        plt.show()  # Mostrar en pantalla

        # Función para matriz de confusión
    def generate_confusion_matrix(model, X, y):
        # Obtener etiquetas verdaderas (índices de mayor probabilidad)
        y_true = np.argmax(y, axis=1)
        # Predecir y obtener índices de clases
        y_pred = np.argmax(model.predict(X), axis=1)
        
        # Calcular matriz de confusión (formato numérico)
        cm = confusion_matrix(y_true, y_pred)
    
        # Crear figura para el heatmap
        plt.figure(figsize=(10, 8))
        # Heatmap con anotaciones numéricas
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=gestures,  # Nombres clases en X
                    yticklabels=gestures)  # Nombres clases en Y
        plt.title('Matriz de Confusión', pad=20)
        plt.xlabel('Predicciones del Modelo')
        plt.ylabel('Etiquetas Reales')
        # Rotar etiquetas X 45 grados para mejor legibilidad
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        # Ajustar márgenes
        plt.tight_layout()
        plt.savefig('confusion_matrix.png', dpi=300)
        plt.show()

        # Reporte detallado de métricas
        print("\nReporte de Clasificación:")
        print(classification_report(y_true, y_pred, 
                                target_names=gestures,
                                digits=3))  # 3 decimales

    # 7. Guardar modelo y resultados
    model.save(model_path)
    generate_training_plots(history)
    generate_confusion_matrix(model, X, y)
    print(f"\nModelo guardado en {model_path}")
    
    # 8. Conversión a TFLite con configuraciones especiales
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False
    
    try:
        tflite_model = converter.convert()
        with open('model_quantized.tflite', 'wb') as f:
            f.write(tflite_model)
        print("\nModelo TFLite exportado exitosamente")
    except Exception as e:
        print(f"\nError en conversión TFLite: {str(e)}")

    
    # Mostrar métricas finales
    val_accuracy = history.history['val_accuracy'][-1]
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## TF LITE

In [37]:
def convert_to_tflite():
    try:
        # Cargar el modelo entrenado
        model = tf.keras.models.load_model(model_path)
        
        # Configurar el conversor con parámetros especiales
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Añadir estas 3 líneas clave para compatibilidad con LSTM
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter._experimental_lower_tensor_list_ops = False
        converter.allow_custom_ops = True  # Permitir operaciones personalizadas
        
        # Realizar la conversión
        tflite_model = converter.convert()
        
        # Guardar el modelo cuantizado
        with open('model_quantized_90.tflite', 'wb') as f:
            f.write(tflite_model)
            
        print("\n✅ Conversión a TFLite exitosa!")
        
    except Exception as e:
        print(f"\n❌ Error en conversión: {str(e)}")
        print("Posibles soluciones:")
        print("1. Verifique que el modelo .h5 existe")
        print("2. Actualice TensorFlow: pip install --upgrade tensorflow")
        print("3. Reinicie el runtime/kernel")

def representative_dataset_gen():
    # Generador de datos de ejemplo para calibración
    for _ in range(100):
        yield [np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)]

## 8. EVALUACION DEL MODELO

In [38]:
from threading import Thread
def evaluate():
    if not os.path.exists("model_quantized_90.tflite"):
        print("\n¡Primero debe entrenar y convertir el modelo!")
        return
    
    # 1. Cargar parámetros y modelo
    try:
        with np.load('normalization_params_90.npz') as data:
            X_mean = data['mean']
            X_std = data['std']
            
        interpreter = tf.lite.Interpreter(model_path="model_quantized_90.tflite")
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]  # Mantener [0]
        output_details = interpreter.get_output_details()[0]
    except Exception as e:
        print(f"\nError crítico: {str(e)}")
        return

    # 2. Configuración de cámara
    cap = cv2.VideoCapture(0)  # Forzar backend DirectShow
    #import time
    #time.sleep(2)  # Permitir inicialización de cámara
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    #cap.set(cv2.CAP_PROP_FPS, 30)
    
    if not cap.isOpened():
        print("\n¡No se puede acceder a la cámara!")
        return

    # 3. Variables de estado
    current_gesture = None  # Variable para almacenar la última seña detectada
    current_confidence = 0.0
    sequence = deque(maxlen=sequence_length)  # Usar deque
    #last_gesture = None
    frame_counter = 0
    #prediction_active = False

    # 4. Bucle principal
    while True:
        # 4.1 Capturar frame
        ret, frame = cap.read()
        if not ret:
            print("\nError en captura de frame")
            break
            
        frame_counter += 1
        
        # 4.2 Procesamiento cada 2 frames
        if frame_counter % 2 != 0:
            continue
            
        # 4.3 Detección de landmarks
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            # 4.4 Extraer landmarks
            landmarks = []
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
            
            # Rellenar si solo hay una mano
            if len(results.multi_hand_landmarks) < 2:
                landmarks += [0.0] * (total_landmarks - len(landmarks))
                
            # El deque ya maneja automáticamente el tamaño máximo gracias a maxlen
            sequence.append(landmarks)  # Esto mantendrá solo los últimos 'sequence_length' elementos
            
            
            # 4.5 Dibujar landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                

                # Añadir verificación de dimensión de entrada
            if input_details['shape'][1] != sequence_length:
                print("\nError: Dimensiones del modelo no coinciden con la configuración")
                return
            
            # 4.6 Realizar predicción cada 1 segundo (30 frames)
            if len(sequence) == sequence_length and frame_counter % 15 == 0:
                try:
                    # Preprocesamiento CORREGIDO
                    seq_array = np.array(sequence)
                    seq_array = (seq_array - X_mean) / (X_std + 1e-7)  # +epsilon para seguridad
                    
                    # Asegurar 3 dimensiones: [batch_size=1, sequence_length, features]
                    input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                    
                    # Debug: Verificar forma
                    
                    print("Forma de entrada:", input_data.shape)  # Debe mostrar (1, 30, 126)

                    # Inferencia
                    interpreter.set_tensor(input_details['index'], input_data)
                    interpreter.invoke()
                    prediction = interpreter.get_tensor(output_details['index'])[0]
                    
                    # Procesar resultados
                    predicted_idx = np.argmax(prediction)
                    confidence = prediction[predicted_idx]
                    
                    if confidence > 0.75:
                        current_gesture = gestures[predicted_idx]  # Actualizar solo si hay alta confianza
                        current_confidence = confidence
                        print("Nueva seña detectada:", current_gesture)
                        
                        
                        cv2.putText(frame, f"{current_gesture} ({confidence:.2%})", 
                            (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2) #el 0,255,0 es verde
                        
                        print("La seña es: ", current_gesture)

                        # Iniciar la reproducción en un hilo separado
                        threading.Thread(target=speak_text, args=(current_gesture,)).start()
                            
                except Exception as e:
                    print(f"Error en predicción: {str(e)}")
                    current_gesture = None  # Resetear si hay error
                    
        if current_gesture:
            cv2.putText(frame, 
              f"{current_gesture} ({current_confidence:.2%})", 
              (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Mostrar frame reducido
        cv2.imshow("Predicciones", cv2.resize(frame, (640, 480)))
        
        
        # 4.8 Salir con ESC
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 5. Limpieza
    cap.release()
    cv2.destroyAllWindows()
    print("\nSistema de evaluación detenido")

### GRAFICOS DE EVALUACION

In [39]:
def evaluate_with_plots():
    # Verificar si el modelo existe
    if not os.path.exists(model_path):
        print("\nPrimero debe entrenar el modelo.")
        return
    
    # Validar parámetros de normalización
    if X_mean is None or X_std is None:
        print("\nERROR: Debe entrenar el modelo primero para obtener los parámetros de normalización")
        return

    # --- CARGAR MODELO Y DATOS ---
    model = load_model(model_path)  # Cargar modelo entrenado desde archivo .h5
    X, y, _ = load_data()  # Obtener datos de entrenamiento/prueba
    y = tf.keras.utils.to_categorical(y)  # Convertir etiquetas a one-hot encoding
    
    # Normalizar datos (igual que durante el entrenamiento)
    X = (X - X_mean) / X_std
    
    # Generar predicciones del modelo
    y_pred = model.predict(X)  # Obtener probabilidades para cada clase
    y_classes = np.argmax(y_pred, axis=1)  # Convertir a clases (índice de mayor probabilidad)
    y_true = np.argmax(y, axis=1)  # Convertir one-hot encoding a índices

    # --- 1. MATRIZ DE CONFUSIÓN ---
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_true, y_classes)  # Calcular matriz numérica
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=gestures, yticklabels=gestures)  # Heatmap con etiquetas
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicciones')
    plt.ylabel('Valores Verdaderos')
    plt.xticks(rotation=45)  # Rotar etiquetas para mejor legibilidad
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # --- 2. REPORTE DE CLASIFICACIÓN ---
    print("\nReporte de Clasificación:")
    # Imprimir métricas detalladas por clase
    print(classification_report(y_true, y_classes, target_names=gestures))

    # --- 3. CURVAS ROC POR CLASE ---
    plt.figure(figsize=(10, 8))
    for i in range(len(gestures)):
        # Calcular curva ROC para cada clase
        fpr, tpr, _ = roc_curve(y_true, y_pred[:,i], pos_label=i)
        roc_auc = auc(fpr, tpr)  # Calcular área bajo la curva
        plt.plot(fpr, tpr, label=f'{gestures[i]} (AUC = {roc_auc:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--')  # Línea de referencia para aleatoriedad
    plt.xlabel('Tasa de Falsos Positivos')
    plt.ylabel('Tasa de Verdaderos Positivos')
    plt.title('Curvas ROC por Clase')
    plt.legend(loc="lower right")
    plt.show()

    # --- 4. PRECISIÓN vs RECALL ---
    plt.figure(figsize=(10, 8))
    for i in range(len(gestures)):
        # Calcular curva Precisión-Recall para cada clase
        precision, recall, _ = precision_recall_curve(y_true, y_pred[:,i], pos_label=i)
        plt.plot(recall, precision, label=gestures[i])
    
    plt.xlabel('Recall')
    plt.ylabel('Precisión')
    plt.title('Curvas Precisión-Recall')
    plt.legend(loc="best")
    plt.show()

    # --- 5. DISTRIBUCIÓN DE PROBABILIDADES ---
    plt.figure(figsize=(10, 6))
    for i in range(len(gestures)):
        # Graficar distribución de probabilidades para predicciones correctas
        sns.kdeplot(y_pred[y_true == i][:,i], label=gestures[i])
    
    plt.xlabel('Probabilidad Predicha')
    plt.ylabel('Densidad')
    plt.title('Distribución de Probabilidades por Clase')
    plt.legend()
    plt.show()

## 9. REENTRENAR GESTO

In [40]:
def retrain_gesture():
    global gestures
    if not gestures:
        print("\nNo hay gestos para reentrenar. Primero recolecte datos.")
        return

    print("\nGestos disponibles para reentrenar:")
    for i, gesture in enumerate(gestures):
        print(f"{i+1}. {gesture}")

    try:
        choice = int(input("\nSeleccione el número del gesto a reentrenar: ")) - 1
        if 0 <= choice < len(gestures):
            gesture = gestures[choice]
            gesture_dir = os.path.join(dataset_dir, gesture)
            
            for file in os.listdir(gesture_dir):
                os.remove(os.path.join(gesture_dir, file))
            
            print(f"\nDatos anteriores de '{gesture}' eliminados.")
            collect_data()
            train_model()
        else:
            print("\nSelección inválida.")
    except ValueError:
        print("\nPor favor, ingrese un número válido.")

## 10. MENU

In [41]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo, y despues ir a convertir a TFlite")
        print("4. Convertir a TFLite")  # Nueva opción
        print("5. Evaluar")
        print("6. Evaluar con Graficos")
        print("7. Reentrenar Gesto")
        print("8. Salir")
        
        choice = input("\nSeleccione una opción: ")
        
        if choice == '1':
            detect_hands()
        elif choice == '2':
            collect_data()
        elif choice == '3':
            train_model()
        elif choice == '4':  # Nueva opción de conversión
            convert_to_tflite()
        elif choice == '5':
            evaluate()
        elif choice == '6':
            evaluate_with_plots()
        elif choice == '7':
            retrain_gesture()
        elif choice == '8':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [42]:
if __name__ == "__main__":
    main()


=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Convertir a TFLite
5. Evaluar
6. Evaluar con Graficos
7. Reentrenar Gesto
8. Salir

Recolectando datos para el gesto '4'. Presiona 'ESC' para cancelar.
Mantenga la seña frente a la cámara...
Secuencias capturadas: 1/3
Secuencias capturadas: 2/3
Secuencias capturadas: 3/3

Se recolectaron 3 secuencias para el gesto '4'

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Convertir a TFLite
5. Evaluar
6. Evaluar con Graficos
7. Reentrenar Gesto
8. Salir

¡Hasta luego!
